# ChatDrug Usage Guide
This is a guide for ChatDrug usage.

## Import Packages

In [1]:
%%capture
import sys
from ChatDrug.task_and_evaluation.Conversational_LLMs_utils import complete
from utils import (
    construct_PDDS_prompt, load_retrieval_DB, retrieve_and_feedback, load_thredhold
)
from ChatDrug.task_and_evaluation import get_task_specification_dict, evaluate, parse

## Conversation Function

In [2]:
def demo_conversation(test_drug, task, C, constraint, retrieval_DB, threshold_dict, messages):
    for round_index in range(C+1):
        generated_text = complete(messages, 'chatgpt')
        messages.append({"role": "assistant", "content": generated_text})
        
        print(">>Start Conversation " + str(round_index+1))
        print("User: " + messages[2*round_index+1]["content"])
        print("ChatGPT: " + generated_text)

        if round_index < 1:
            closest_drug = None
        generated_drug_list = parse(task, test_drug, generated_text, closest_drug)

        # Check Parsing Results
        if generated_drug_list == None:
            print(">>Invalid Drug Generation")
            break
        elif len(generated_drug_list) == 0:
            print(">>Drug Generation Error")
            break
        else:
            generated_drug = generated_drug_list[0]
            print("Generated Result: "+str(generated_drug))
        
        # Check Evaluation Results
        answer = evaluate(test_drug, generated_drug, task, constraint, threshold_dict)

        if answer == -1:
            print(">>Invalid Drug Generation")
            break

        print('Evaluation result: '+str(answer))

        if answer:
            print(">>Drug Generation Correct")
            break
        else:
            if round_index < C:
                print('>>Start Retrieval ' + str(round_index+1))
                try:
                    closest_drug = retrieve_and_feedback(task, retrieval_DB, test_drug, generated_drug, constraint, threshold_dict)
                except:
                    error = sys.exc_info()
                    if error[0] == Exception:
                        print('>>Cannot Find Retrieval Result')
                        break
                    else:
                        print('>>Invalid Drug Generation')
                        break

                print("Retrieval Result: " + closest_drug)

                prompt_ReDF = f'Your provided sequence {generated_drug} is not correct. We find a sequence {closest_drug} which is correct and similar to the {test_drug_type} you provided. Can you give me a new {test_drug_type}?'
                messages.append({"role": "user", "content": prompt_ReDF})
            else:
                print(">>Drug Generation Error")

## Parameters
The following blocks are ChatDrug parameters for molecule, peptide and protein respectively. Choose the drug type by running corresponding blocks. 
`test_drug`, `task` and `C` can also be changed by setting your own input durg, task id and conversation round number. Here are several examples:

### Molecule

In [3]:
test_drug_type = 'molecule'
test_drug = 'O=C(NC[C@H]1CCCO1)c1ccccc1N1CCCC1=O'
task = 203
C = 2
constraint = 'loose'

### Peptide

In [ ]:
test_drug_type = 'peptide'
test_drug = 'AAASHFFNL'
task = 301
C = 2 
constraint = None

### Peotein

In [ ]:
test_drug_type = 'protein'
test_drug = 'GAPVPVDENDEGLQRALQFAMAEYNRASNDKYSSRVVRVISAKRQLVSGIKYILQVEIGRTTCPKSSGDLQSCEFHDEPEMAKYTTCTFVVYSIPWLNQIKLLESKCQ'
task = 501
C = 2
constraint = None

## Load Data

In [4]:
task_specification_dict = get_task_specification_dict(task)
input_drug_list, retrieval_DB = load_retrieval_DB(task, seed=0)
threshold_dict = load_thredhold(test_drug_type)

## Start ChatDrug

In [5]:
print(">>Start ChatDrug")

# ChatGPT message
messages = [{"role": "system", "content": "You are an expert in the field of molecular chemistry."}]
print("System: You are an expert in the field of molecular chemistry.")

# PDDS
PDDS_prompt = construct_PDDS_prompt(task_specification_dict, test_drug, test_drug_type, task)
messages.append({"role": "user", "content": PDDS_prompt})

# Conversation
demo_conversation(test_drug, task, C, constraint, retrieval_DB, threshold_dict, messages)

>>Start ChatDrug
System: You are an expert in the field of molecular chemistry.
>>Start Conversation 1
User: Can you make molecule O=C(NC[C@H]1CCCO1)c1ccccc1N1CCCC1=O more soluble in water and more hydrogen bond donors? The output molecule should be similar to the input molecule. Give me five molecules in SMILES only and list them using bullet points. No explanation is needed.
ChatGPT: Sure, here are five molecules in SMILES that are similar to the input molecule but have increased solubility in water and more hydrogen bond donors:

- O=C(NC[C@H]1CCCO1)c1ccccc1N1CCOCC1=O
- O=C(NC[C@H]1CCCO1)c1ccccc1N1CCNCC1=O
- O=C(NC[C@H]1CCCO1)c1ccccc1N1CC(O)CC1=O
- O=C(NC[C@H]1CCCO1)c1ccccc1N1CCC(O)C1=O
- O=C(NC[C@H]1CCCO1)c1ccccc1N1CC(O)(C)C1=O
Generated Result: O=C(NC[C@H]1CCCO1)c1ccccc1N1CCOCC1=O
Evaluation result: False
>>Start Retrieval 1
Retrieval Result: O=C(NC[C@@H]1CCCO1)C(=O)Nc1cnc(-c2ccccc2)s1
>>Start Conversation 2
User: Your provided sequence O=C(NC[C@H]1CCCO1)c1ccccc1N1CCOCC1=O is not 